# Вариант 4. Сравнение моделей нейронов

Реализация трёх моделей: **LIFNeuron** (Leaky Integrate-and-Fire), **IzhikevichNeuron**, **SimpleIntegrateNeuron**. Для каждой — метод `step` с соответствующими уравнениями. Сравнение реакции на один и тот же входной ток (ступенька, импульс, синусоида) и графики мембранного потенциала и спайковой активности.

In [ ]:
# Импорты
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt

## 1. Модели нейронов

У каждой модели метод **`step(dt, I)`** возвращает текущий мембранный потенциал и флаг спайка (после обновления состояния).

In [ ]:
class LIFNeuron:
    """
    Leaky Integrate-and-Fire нейрон.
    Уравнение: tau * dV/dt = -(V - V_rest) + R*I; при V >= V_th — сброс в V_reset.
    """

    def __init__(
        self,
        V_rest: float = -70.0,
        V_th: float = -55.0,
        V_reset: float = -70.0,
        tau: float = 10.0,
        R: float = 1.0,
    ) -> None:
        self.V_rest = V_rest
        self.V_th = V_th
        self.V_reset = V_reset
        self.tau = tau
        self.R = R
        self.V = float(V_rest)

    def step(self, dt: float, I: float) -> tuple[float, bool]:
        """Шаг по времени: dV/dt = (R*I - (V - V_rest)) / tau; при V >= V_th — спайк и сброс."""
        self.V += dt * (self.R * I - (self.V - self.V_rest)) / self.tau
        spike = self.V >= self.V_th
        if spike:
            self.V = self.V_reset
        return self.V, spike


class IzhikevichNeuron:
    """
    Нейрон Ижикевича (двумерная модель).
    dV/dt = 0.04*V^2 + 5*V + 140 - u + I; du/dt = a*(b*V - u); при V >= 30 — сброс V=c, u+=d.
    """

    def __init__(
        self,
        a: float = 0.02,
        b: float = 0.2,
        c: float = -65.0,
        d: float = 8.0,
    ) -> None:
        self.a, self.b, self.c, self.d = a, b, c, d
        self.V = -65.0
        self.u = self.b * self.V

    def step(self, dt: float, I: float) -> tuple[float, bool]:
        """Один шаг Эйлера для V и u; при V >= 30 — спайк и сброс."""
        self.V += dt * (0.04 * self.V**2 + 5 * self.V + 140 - self.u + I)
        self.u += dt * self.a * (self.b * self.V - self.u)
        spike = self.V >= 30.0
        if spike:
            self.V = self.c
            self.u += self.d
        return self.V, spike


class SimpleIntegrateNeuron:
    """
    Простой интегратор (Integrate-and-Fire без утечки).
    dV/dt = I/C; при V >= V_th — сброс в V_reset.
    """

    def __init__(
        self,
        V_th: float = 5.0,
        V_reset: float = 0.0,
        C: float = 1.0,
    ) -> None:
        self.V_th = V_th
        self.V_reset = V_reset
        self.C = C
        self.V = 0.0

    def step(self, dt: float, I: float) -> tuple[float, bool]:
        """V += I*dt/C; при V >= V_th — спайк и сброс."""
        self.V += I * dt / self.C
        spike = self.V >= self.V_th
        if spike:
            self.V = self.V_reset
        return self.V, spike

## 2. Входные токи и запуск симуляции

Три типа входа: **ступенька** (постоянный ток после t0), **импульс** (короткий прямоугольный импульс), **синусоида**.

In [ ]:
def run_neuron(neuron_factory, I_signal: np.ndarray, t: np.ndarray, dt: float) -> tuple[np.ndarray, np.ndarray]:
    """Запускает нейрон на токе I_signal; возвращает V и массив времён спайков."""
    n = len(t)
    V = np.zeros(n)
    spikes = []
    neuron = neuron_factory()
    for i in range(n):
        V[i], sp = neuron.step(dt, I_signal[i])
        if sp:
            spikes.append(t[i])
    return V, np.array(spikes) if spikes else np.array([])


# Время и шаг
dt = 0.1
t = np.arange(0, 200, dt)
n = len(t)

# Ступенька: I = I0 при t >= t0
t0_step, I0_step = 20, 15.0
I_step = np.where(t >= t0_step, I0_step, 0.0)

# Импульс: I = I0 на [t1, t2]
t1_imp, t2_imp, I0_imp = 20, 30, 25.0
I_imp = np.where((t >= t1_imp) & (t <= t2_imp), I0_imp, 0.0)

# Синусоида: I = I_bias + A*sin(2*pi*f*t)
I_bias, A_sin, f_sin = 10.0, 8.0, 0.05
I_sin = I_bias + A_sin * np.sin(2 * np.pi * f_sin * t)

## 3. Графики: мембранный потенциал и спайковая активность

Для каждого типа входа — один рисунок: сверху потенциалы V(t) для трёх моделей, снизу — растр спайков (или отмеченные моменты спайков).

In [ ]:
MODEL_ORDER = ["LIF", "Izhikevich", "SimpleIntegrate"]


def plot_response(t: np.ndarray, results: dict[str, tuple[np.ndarray, np.ndarray]], title: str) -> None:
    """Строит V(t) и растр спайков для трёх моделей."""
    fig, (ax_v, ax_sp) = plt.subplots(2, 1, figsize=(10, 5), sharex=True, height_ratios=[1.5, 0.6])
    colors = {"LIF": "C0", "Izhikevich": "C1", "SimpleIntegrate": "C2"}
    for name in MODEL_ORDER:
        V, sp_t = results[name]
        ax_v.plot(t, V, label=name, color=colors[name], alpha=0.9)
        if len(sp_t) > 0:
            y_pos = MODEL_ORDER.index(name)
            ax_sp.scatter(sp_t, np.full_like(sp_t, y_pos), color=colors[name], s=6, marker="|")
    ax_v.set_ylabel("V (потенциал)")
    ax_v.legend(loc="upper right")
    ax_v.set_title(title)
    ax_v.grid(True, alpha=0.3)
    ax_sp.set_ylabel("модель")
    ax_sp.set_yticks([0, 1, 2])
    ax_sp.set_yticklabels(MODEL_ORDER)
    ax_sp.set_xlabel("t")
    ax_sp.set_ylim(-0.5, 2.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Ступенька тока
res_step = {
    "LIF": run_neuron(LIFNeuron, I_step, t, dt),
    "Izhikevich": run_neuron(IzhikevichNeuron, I_step, t, dt),
    "SimpleIntegrate": run_neuron(SimpleIntegrateNeuron, I_step, t, dt),
}
plot_response(t, res_step, "Вход: ступенька тока (I = const после t=20)")

In [ ]:
# Импульс тока
res_imp = {
    "LIF": run_neuron(LIFNeuron, I_imp, t, dt),
    "Izhikevich": run_neuron(IzhikevichNeuron, I_imp, t, dt),
    "SimpleIntegrate": run_neuron(SimpleIntegrateNeuron, I_imp, t, dt),
}
plot_response(t, res_imp, "Вход: импульс тока (прямоугольник 20–30)")

In [ ]:
# Синусоидальный ток
res_sin = {
    "LIF": run_neuron(LIFNeuron, I_sin, t, dt),
    "Izhikevich": run_neuron(IzhikevichNeuron, I_sin, t, dt),
    "SimpleIntegrate": run_neuron(SimpleIntegrateNeuron, I_sin, t, dt),
}
plot_response(t, res_sin, "Вход: синусоида I = I_bias + A*sin(2πft)")